In [2]:
import cv2
import matplotlib.pyplot as plt

import os
import numpy as np

In [9]:
files=os.listdir()
d={}
cnt=0

X=[]
Y=[]

for f in files:
    if f.endswith(".npy"): 
        if f.startswith("X")==True&f.startswith("Y")!=True:
            data=np.load(f)
            print(data.shape,f)
            labels=np.ones((data.shape[0],1))*cnt
            
            
            X.append(data)
            Y.append(labels)
            d[cnt]=f[:-4]
            cnt+=1

(20, 100, 100, 3) abhilasha.npy
(20, 100, 100, 3) manali.npy
(20, 100, 100, 3) shreeya.npy


In [10]:
X=np.vstack(X)
Y=np.vstack(Y)
Y=Y.reshape((60,1))

In [11]:
print(X.shape,Y.shape)

(60, 100, 100, 3) (60, 1)


In [12]:
def dist(v1,v2):#point can be n dimension
    return np.sqrt(np.sum((v1-v2)**2))

def knn(X,Y,query_point,k=7):
    m=X.shape[0]
    distances=[]
    for i in range(m):
        distances.append((dist(query_point,X[i]),Y[i]))
        
    distances=sorted(distances)[:5]
    distances=np.array(distances)
    labels=distances[:,1]
    labels,freq=np.unique(labels,return_counts=True)
    pred=labels[np.argmax(freq)]
    
    return int(pred)

In [13]:
cam=cv2.VideoCapture(0)
detector=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

while True:
    ret,frame=cam.read()
    
    if ret== False:
        continue
    
    faces=detector.detectMultiScale(frame,1.2)
    if(len(faces)==0):
        continue
        
    for face in faces:
      
        x,y,w,h=face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
           
        cropped_face=frame[y:y+h,x:x+w]
        cropped_face=cv2.resize(cropped_face,(100,100))
        pred=knn(X,Y,cropped_face,k=5)
        pred_name=d[pred]
        cv2.putText(frame, pred_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), lineType=cv2.LINE_AA)
    cv2.imshow("frame",frame)
    key_pressed=cv2.waitKey(1) & 0xFF
    if(key_pressed==ord('q')):
        break
    
cam.release()
cv2.destroyAllWindows()